In [ ]:
import pandas as pd
# from sec_edgar_api import EdgarClient
# from sec_edgar_downloader import Downloader
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np
import glob
import logging
from typing import Literal

## Read HTML File

In [ ]:
def read_html(file: str):
    """Reads html file and returns BeautifulSoup object"""
    with open(f'{file}', 'r') as f:
        contents = f.read()
    soup = BeautifulSoup(contents, 'html.parser')
    return soup

## Find target table

In [ ]:
def find_target_table(target_text, soup, search_type=Literal['loose','strict']):
    """
    target_text: regex pattern
    
    soup: BeautifulSoup object
    
    search_type: 'loose' or 'strict'
        - loose: search for target_text in the soup
        - strict: search for target_text in the soup, then find the next table following the text

    Returns a list of BeautifulSoup objects containing the target table(s) in html format.
    """
    target_tables = None
    
    target_element = soup.find_all('div') # find_all returns a list
    if search_type == 'loose':
        target_tables = [i for i in target_element if target_text.search(i.text.lower())]
    elif search_type == 'strict':
        target_tables = [j.find_next('table') for j in [i for i in target_element if target_text.search(i.text)]]
    
    return target_tables

## Extract Table from HTML

In [ ]:
def extract_table_from_html(table_html):
    """
    table_html: BeautifulSoup object
    
    Returns a dataframe of the table.
    
    """
    data = [] # row x col [[1,2,3,4,5], [1,2,3,4,5], [1,2,3,4,5]] = 3 rows, 5 columns
    # Loop over each row
    for row in table_html.find_all('tr')[:]: #
        # print(f'row: {row}')
        cols = []
        # Loop over each cell in the row
        for cell in row.find_all(['td', 'th'])[:]:
            # print(cell)
            # Get the colspan and rowspan attributes, defaulting to 1 if they don't exist
            colspan = int(cell.get('colspan', 1))
            rowspan = int(cell.get('rowspan', 1))

            # print(f'cell: {cell.text.strip()}\ncolspan: {colspan}\nrowspan: {rowspan}')
            # If the cell spans multiple rows or columns, add copies of it to the cols list
            for i in range(rowspan):
                for j in range(colspan):
                    cols.append(cell.text.strip())


        # Add the cols list to the data list
        data.append(cols)
        # print(data)
    df = pd.DataFrame(data)
    df.replace('', np.nan, inplace=True)
    return df

## Clean df

In [ ]:
def clean_df(df):
    """
    df: dataframe
    
    Process the dataframe to remove unnecessary rows and columns.
    
    Steps:
    1. Find header row and use it to set column names
    2. Remove non-header rows and columns where all values are NaN
    3. Replace empty strings with NaN
    4. Forward fill NaN values along columns
    5. Remove rows where all values are NaN
    6. Reset index after dropping rows
    7. if column contains '$' then remove it
    8. if row only contains same element then store the index and value in a dictionary
    9. Use the dictionary to assign the value as first level of multiindex for the rows in between the rows to keep
    10. replace any cell that contains : with empty string
    11. Combine two columns to create a multiindex for the rows

    Returns a cleaned dataframe.
    """

    first_row = df.iloc[:, 0].notnull().idxmax()
    # Use first three rows for the header
    header = df.iloc[0:first_row].fillna('').agg(' '.join).str.strip()
    df.columns = header

    # Remove the rows used for header and reset index
    df = df.iloc[first_row:].reset_index(drop=True)

    # Remove columns where all values are NaN
    df = df.dropna(how='all', axis=1)

    # Replace empty strings with NaN
    df.replace("", np.nan, inplace=True)

    # Forward fill NaN values along columns
    df.fillna(method='ffill', axis=1, inplace=True)

    # Remove rows where all values are NaN
    df = df.dropna(how='all', axis=0)

    # Reset index after dropping rows
    df.reset_index(drop=True, inplace=True)

    # if column contains '$' then remove it
    col_to_keep = [i for i,x in enumerate(df.columns) if '$' not in df.iloc[:, i].values and '%' not in df.iloc[:, i].values and 'change' not in x.lower()]
    df = df.iloc[:, col_to_keep]
    df = df.loc[:, ~df.columns.duplicated(keep='first')]

    # if row only contains same element then store the index and value in a dictionary
    row_to_keep = {}
    for i in range(len(df)):
        if len(set(df.iloc[i, :])) == 1:
            row_to_keep[i] = df.iloc[i, 0]

    # Use the dictionary to assign the value as first level of multiindex for the rows in between the rows to keep
    df['Category'] = df.iloc[:, 0]
    for i in range(len(df)):
        if i in row_to_keep.keys():
            df.iloc[i, -1] = row_to_keep[i]
        else:
            df.iloc[i, -1] = df.iloc[i-1, -1]


    row_to_keep = []
    for i in range(len(df)):
        if len(set(df.iloc[i, :])) != 1:
            row_to_keep.append(i)

    df = df.iloc[row_to_keep, :]
    multiindex = df.iloc[:,[-1,0]]

    # replace any cell that contains : with empty string
    df = df.replace(to_replace=':', value='', regex=True)

    # Combine two columns to create a multiindex for the rows
    df.index = pd.MultiIndex.from_arrays([df.iloc[:,-1].values, df.iloc[:,0].values])
    df = df.iloc[:, 1:-1]
    return df

In [ ]:
# Configure the logging settings
logging.basicConfig(level=logging.DEBUG, filename='loop_logs.log', filemode='w', format='%(asctime)s - %(levelname)s - %(message)s')

In [ ]:
file_to_read = html_files[6]
file_name = file_to_read.split("\\")[-2]
print(f'Reading html files... {file_name}')
html_soup = read_html(file_to_read)
target_table = find_target_table(re.compile(r'.*(Products and services|Net sales|Sales Data|net sales by operating segment).*'), html_soup, search_type='strict')

In [ ]:
df = extract_table_from_html(target_table[4])
df = clean_df(df)
df = df.stack(level=0).reset_index(level=2).reset_index()
df.columns = ['Category', 'Segment', 'Date_info', 'Value']
df.to_dict('records')
df

In [ ]:
all_df = pd.DataFrame()
number_of_files = len(html_files)
for file in html_files[:]:
    logging.info(file)
    file_name = file.split("\\")[-2]
    
    try:
        html_soup = read_html(file)
        logging.info(f'Step 1: {file_name}: Read html successfully.')
    
        target_table = find_target_table(re.compile(r'.*(Sales Data|net sales by operating segment|net sales|products and services).*'), html_soup, search_type='strict')
        logging.info(f'Step 2: {file_name}: Found target table successfully.')

        assert target_table is not None, f'{file_name}: Target table not found.'
        df = extract_table_from_html(target_table).iloc[:, :]
        logging.info(f'Step 3: {file_name}: Extracted table successfully.')

        df = clean_df(df)
        logging.info(f'Step 4: {file_name}: Cleaned table successfully.')

        df = df.stack(level=0).reset_index(level=2).reset_index()
        logging.info(f'Step 5: {file_name}: Stacked table successfully.')
    
        df.columns = ['Category', 'Segment', 'Date_info', 'Value']
        logging.info(f'Step 6: {file_name}: Renamed columns successfully.')
    
        all_df = pd.concat([all_df, df], axis=0, ignore_index=True)
        logging.info(f'Step 7: {file_name}: Concatenated table successfully.')
    except Exception as e:
        logging.info(f'Step 7: {file_name}: Concatenated table unsuccessfully.')
        logging.error(e)
    
    number_of_files -= 1
    logging.info(f'Number of files left: {number_of_files}')
    break
all_df['Date'] = all_df['Date_info'].str.extract(r"([A-Za-z]+\s\d{1,2},\s\d{4})")

# Extract three months ended text from date column
all_df['Time Leading To'] = all_df['Date_info'].str.extract(r"([A-Za-z]+\s[A-Za-z]+\s[A-Za-z]+)")

all_df.drop(columns=['Date_info'], inplace=True)

# reorder columns
all_df = all_df[['Category', 'Segment', 'Time Leading To', 'Date', 'Value']]
all_df.to_csv('all_df.csv', index=False)

In [ ]:
html_soup = read_html(html_files[6])
target_text = re.compile(r'.*(sales data|net sales|products).*')
target_element = html_soup.find_all('div')
# all_text = [i for i in target_element if target_text.search(i.text.lower())]
all_text = [j.find_next('table') for j in [i for i in target_element if target_text.search(i.text)]]
len(all_text)

In [ ]:
for i in all_text[:22]:
    table = extract_table_from_html(i)
    if len(table) != 0:
        actual = table
df = clean_df(actual)
df = df.stack(level=0).reset_index(level=2).reset_index()

df.columns = ['Category', 'Segment', 'Date_info', 'Value']
df

In [ ]:
df.to_dict(orient='records')

# Using Pandas

In [ ]:
sales_data_table_02 = pd.read_html(r"sec-edgar-filings\0000320193\10-Q\0000912057-02-004945\filing-details.html")
sales_data_table_23 = pd.read_html(r"sec-edgar-filings\0000320193\10-Q\0000320193-23-000006\filing-details.html")
sales_data_table_15 = pd.read_html(r"sec-edgar-filings\0000320193\10-Q\0001193125-15-259935\filing-details.html")

In [ ]:
sales_data_table_15[38]

In [ ]:
df = sales_data_table_15[38]
# Use first three rows for the header
header = df.iloc[0:3].fillna('').agg(' '.join).str.strip()
df.columns = header

# Remove the rows used for header and reset index
df = df.iloc[3:].reset_index(drop=True)

# Remove columns where all values are NaN
df = df.dropna(how='all', axis=1)

# Replace empty strings with NaN
df.replace("", np.nan, inplace=True)

# Forward fill NaN values along columns
df.fillna(method='ffill', axis=1, inplace=True)

# Remove rows where all values are NaN
df = df.dropna(how='all', axis=0)

# Reset index after dropping rows
df.reset_index(drop=True, inplace=True)

# if column contains '$' then remove it
col_to_keep = [i for i,x in enumerate(df.columns) if '$' not in df.iloc[:, i].values]
df = df.iloc[:, col_to_keep]
df = df.loc[:, ~df.columns.duplicated(keep='first')]

# if row only contains same element then store the index and value in a dictionary
row_to_keep = {}
for i in range(len(df)):
    if len(set(df.iloc[i, :])) == 1:
        row_to_keep[i] = df.iloc[i, 0]

# Use the dictionary to assign the value as first level of multiindex for the rows in between the rows to keep
df['Category'] = df.iloc[:, 0]
for i in range(len(df)):
    if i in row_to_keep.keys():
        df.iloc[i, -1] = row_to_keep[i]
    else:
        df.iloc[i, -1] = df.iloc[i-1, -1]


row_to_keep = []
for i in range(len(df)):
    if len(set(df.iloc[i, :])) != 1:
        row_to_keep.append(i)

df = df.iloc[row_to_keep, :]
multiindex = df.iloc[:,[-1,0]]

# replace any cell that contains : with empty string
df = df.replace(to_replace=':', value='', regex=True)

# Combine two columns to create a multiindex for the rows
df.index = pd.MultiIndex.from_arrays([df.iloc[:,-1].values, df.iloc[:,0].values])
df = df.iloc[:, 1:-1]

# unpivot the dataframe based on the third column
df = df.stack(level=0).reset_index(level=2).reset_index()
df


# Find a way to convert data in the tables to a key-value pair in JSON format

In [ ]:
# convert df to dictionary and label with AAPL
df = df.to_dict(orient='index')
df = {'AAPL': df}
df # shares in thousands, eps in per share, and others in millions

# Test SEC API

In [71]:
import requests
import pandas as pd
import json
import xml.etree.ElementTree as ET
from ratelimit import limits, sleep_and_retry
from bs4 import BeautifulSoup
from tqdm import trange
import logging
import re

class SECData:
    """Class to retrieve data from SEC Edgar database.

    Args:
        requester_name (str): Name of the requester
        requester_email (str): Email of the requester
        taxonomy (str): us-gaap, ifrs-full, dei, or srt

    Raises:
        Exception: If taxonomy is not one of the following: us-gaap, ifrs-full, dei, or srt

    Attributes:
        BASE_API_URL (str): Base url for SEC Edgar database
        US_GAAP_TAXONOMY_URL (str): URL for us-gaap taxonomy
        ALLOWED_TAXONOMIES (list): List of allowed taxonomies
        headers (dict): Headers to be used for API calls
        cik (DataFrame): DataFrame containing CIK and ticker
        tags (list): List of tags in us-gaap taxonomy
        taxonomy (str): us-gaap, ifrs-full, dei, or srt

    Methods:
        get_cik_list: Retrieves the full list of CIK available from SEC database.
        get_ticker_cik: Get a specific ticker's CIK number. 
        get_usgaap_tags: Get the list of tags in us-gaap taxonomy.
        get_submissions: Retrieves the list of submissions for a specific CIK.
        get_company_concept: Retrieves the XBRL disclosures from a single company (CIK) 
            and concept (a taxonomy and tag) into a single JSON file.
        get_company_facts: Retrieves the XBRL disclosures from a single company (CIK) 
            into a single JSON file.
        get_frames: Retrieves one fact for each reporting entity that is last filed that most closely fits the calendrical period requested.
    """

    BASE_API_URL = "https://data.sec.gov/"
    BASE_SEC_URL = "https://www.sec.gov/"
    BASE_DIRECTORY_URL = "https://www.sec.gov/Archives/edgar/data/"
    SIC_LIST_URL = "https://www.sec.gov/corpfin/division-of-corporation-finance-standard-industrial-classification-sic-code-list"
    US_GAAP_TAXONOMY_URL = "https://xbrl.fasb.org/us-gaap/2023/elts/us-gaap-2023.xsd"
    ALLOWED_TAXONOMIES = ['us-gaap', 'ifrs-full', 'dei', 'srt']
    INDEX_EXTENSION = ['-index.html', '-index-headers.html']
    FILE_EXTENSIONS = ['.xsd', '.htm', '_cal.xml',
                       '_def.xml', '_lab.xml', '_pre.xml', '_htm.xml', '.xml']

    def __init__(self, requester_company: str = 'Financial API', requester_name: str = 'API Caller', requester_email: str = 'apicaller@gmail.com', taxonomy: str = 'us-gaap',):
        # Initialize logger
        self.scrape_logger = logging.getLogger('sec_scraper')
        self.scrape_logger.setLevel(logging.DEBUG)

        # Check if the self.scrape_logger already has handlers to avoid duplicate logging.
        if not self.scrape_logger.hasHandlers():
            # Create a file handler
            file_handler = logging.FileHandler(
                r'C:\Users\lianz\Python\finance-dashboard\utils\sec-scraper\logs\sec_logs.log')
            file_handler.setLevel(logging.DEBUG)

            # Create a stream handler
            stream_handler = logging.StreamHandler()
            stream_handler.setLevel(logging.DEBUG)

            # Create a logging format
            formatter = logging.Formatter(
                '%(asctime)s - %(name)s - %(levelname)s - %(message)s')
            file_handler.setFormatter(formatter)
            stream_handler.setFormatter(formatter)

            # Add the handlers to the self.scrape_logger
            self.scrape_logger.addHandler(file_handler)
            self.scrape_logger.addHandler(stream_handler)

        self.requester_company = requester_company
        self.requester_name = requester_name
        self.requester_email = requester_email
        self.sec_headers = {"User-Agent": f"{requester_company} {requester_name} {requester_email}",
                            "Accept-Encoding": "gzip, deflate",
                            "Host": "www.sec.gov"}
        self.sec_data_headers = {"User-Agent": f"{requester_company} {requester_name} {requester_email}",
                                 "Accept-Encoding": "gzip, deflate",
                                 "Host": "data.sec.gov"}
        self._cik_list = None
        self._tags = None
        if taxonomy not in self.ALLOWED_TAXONOMIES:
            raise ValueError(
                f"Taxonomy {taxonomy} is not supported. Please use one of the following taxonomies: {self.ALLOWED_TAXONOMIES}")
        self.taxonomy = taxonomy

    @property
    def cik_list(self,):
        if self._cik_list is None:
            self._cik_list = self.get_cik_list()
        return self._cik_list
    
    @property
    def tags(self,):
        if self._tags is None:
            self._tags = self.get_usgaap_tags()
        return self._tags

    @sleep_and_retry
    @limits(calls=10, period=1)
    def rate_limited_request(self, url: str, headers: dict):
        """Rate limited request to SEC Edgar database.

        Args:
            url (str): URL to retrieve data from
            headers (dict): Headers to be used for API calls

        Returns:
            response: Response from API call
        """
        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            self.scrape_logger.error(f'''Request failed at URL: {url}''')
        else:
            self.scrape_logger.info(f'''Request successful at URL: {url}''')
        return response

    def get_cik_list(self):
        """Retrieves the full list of CIK available from SEC database.

        Raises:
            Exception: On failure to retrieve CIK list

        Returns:
            cik_df: DataFrame containing CIK and ticker
        """
        url = r"https://www.sec.gov/files/company_tickers.json"
        cik_raw = self.rate_limited_request(url, self.sec_headers)
        cik_json = cik_raw.json()
        cik_df = pd.DataFrame.from_dict(cik_json).T
        return cik_df

    def get_ticker_cik(self, ticker: str,):
        """Get a specific ticker's CIK number. 
        CIK########## is the entity's 10-digit Central Index Key (CIK).

        Args:
            ticker (str): public ticker symbol of the company

        Returns:
            cik: CIK number of the company excluding the leading 'CIK'
        """
        ticker_cik = self.cik_list.query(f"ticker == '{ticker.upper()}'")['cik_str']
        cik = f"{ticker_cik.iloc[0]:010d}"
        return cik

    def get_usgaap_tags(self, xsd_url: str = US_GAAP_TAXONOMY_URL):
        """Get the list of tags (elements) in us-gaap taxonomy or provide a different xsd_url to get tags from a different taxonomy.

        Returns:
            list of tags
        """
        response = self.rate_limited_request(xsd_url, headers=self.sec_headers)
        xsd_content = response.text
        root = ET.fromstring(xsd_content)

        return [element.attrib['name'] for element in root.findall(".//{http://www.w3.org/2001/XMLSchema}element")]

    def get_submissions(self, cik: str = None, submission_file: str = None) -> dict:
        if cik is not None:
            url = f"{self.BASE_API_URL}submissions/CIK{cik}.json"
        elif submission_file is not None:
            url = f"{self.BASE_API_URL}submissions/{submission_file}"
        else:
            raise Exception(
                "Please provide either a CIK number or a submission file.")
        response = self.rate_limited_request(
            url, headers=self.sec_data_headers)
        if response.status_code != 200:
            raise Exception(
                f"Failed to retrieve submissions. Status code: {response.status_code}")
        data = json.loads(response.text)
        return data

    def get_company_concept(self, cik: str, tag: str, taxonomy: str = 'us-gaap',):
        """The company-concept API returns all the XBRL disclosures from a single company (CIK) 
        and concept (a taxonomy and tag) into a single JSON file, with a separate array of facts 
        for each units on measure that the company has chosen to disclose 
        (e.g. net profits reported in U.S. dollars and in Canadian dollars).

        Args:
            cik (str): CIK number of the company. Get the list using self.cik
            taxonomy (str): us-gaap, ifrs-full, dei, or srt
            tag (str): taxonomy tag (e.g. Revenue, AccountsPayableCurrent). See full list from https://xbrl.fasb.org/us-gaap/2023/elts/us-gaap-2023.xsd

        Raises:
            Exception: On failure to retrieve company concept either due to invalid CIK, taxonomy, or tag

        Returns:
            data: JSON file containing all the XBRL disclosures from a single company (CIK)
        """
        url = f"{self.BASE_API_URL}api/xbrl/companyconcept/CIK{cik}/{taxonomy}/{tag}.json"
        response = self.rate_limited_request(
            url, headers=self.sec_data_headers)
        data = json.loads(response.text)
        return data

    def get_company_facts(self, cik):
        url = f"{self.BASE_API_URL}api/xbrl/companyfacts/CIK{cik}.json"
        response = self.rate_limited_request(
            url, headers=self.sec_data_headers)
        if response.status_code != 200:
            raise Exception(
                f"Failed to retrieve company facts for CIK {cik}. Status code: {response.status_code}")
        data = json.loads(response.text)
        return data

    def get_frames(self, taxonomy, tag, unit, period):
        """The xbrl/frames API aggregates one fact for each reporting entity that is last filed that most closely fits the calendrical period requested. 
        This API supports for annual, quarterly and instantaneous data: https://data.sec.gov/api/xbrl/frames/us-gaap/AccountsPayableCurrent/USD/CY2019Q1I.json

        Args:
            taxonomy (str): us-gaap, ifrs-full, dei, or srt
            tag (str): taxonomy tag (e.g. Revenue, AccountsPayableCurrent). See full list from https://xbrl.fasb.org/us-gaap/2023/elts/us-gaap-2023.xsd
            unit (str): USD, USD-per-shares, etc.
            period (str): CY#### for annual data (duration 365 days +/- 30 days), CY####Q# for quarterly data (duration 91 days +/- 30 days), CY####Q#I for instantaneous data

        Raises:
            Exception: (placeholder)

        Returns:
            data: json formatted response
        """
        url = f"{self.BASE_API_URL}api/xbrl/frames/{taxonomy}/{tag}/{unit}/{period}.json"
        response = self.rate_limited_request(
            url, headers=self.sec_data_headers)
        data = json.loads(response.text)
        return data

    def get_data_as_dataframe(self, cik: str,):
        """Retrieves the XBRL disclosures from a single company (CIK) and returns it as a pandas dataframe.

        Args:
            cik (str): CIK number of the company. Get the list using self.cik

        Returns:
            df: pandas dataframe containing the XBRL disclosures from a single company (CIK)
        """
        data = self.get_company_facts(cik)

        df = pd.DataFrame()

        for tag in data['facts'][self.taxonomy]:
            facts = data['facts']['us-gaap'][tag]['units']
            unit_key = list(facts.keys())[0]
            temp_df = pd.DataFrame(facts[unit_key])
            temp_df['label'] = tag
            df = pd.concat([df, temp_df], axis=0, ignore_index=True)
        df = df.astype({'val': 'float64',
                        'end': 'datetime64[ns]',
                        'start': 'datetime64[ns]',
                        'filed': 'datetime64[ns]'})
        df['Months Ended'] = (df['end'] - df['start']
                              ).dt.days.div(30.4375).round(0)
        return df

    def get_index(self, cik: str = None,) -> dict:
        """Each CIK directory and all child subdirectories contain three files to assist in 
        automated crawling of these directories. 
        These are not visible through directory browsing.
            - index.html (the web browser would normally receive these)
            - index.xml (a XML structured version of the same content)
            - index.json (a JSON structured vision of the same content)

        Args:
            cik (str): CIK number of the company. Get the list using self.cik

        Returns:
            json: pandas dataframe containing the XBRL disclosures from a single company (CIK)
        """
        if cik is not None:
            url = self.BASE_DIRECTORY_URL + cik + '/' + 'index.json'

        else:
            url = self.BASE_DIRECTORY_URL + self.cik + '/' + 'index.json'

        response = self.rate_limited_request(url, headers=self.sec_headers)
        return response.json()

    def get_sic_list(self, sic_list_url: str = SIC_LIST_URL) -> dict:
        """Get the list of SIC codes from SEC website.

        Args:
            sic_list_url (str): URL to the list of SIC codes

        Returns:
            pd.DataFrame: DataFrame containing the SIC codes and descriptions
        """
        response = self.rate_limited_request(sic_list_url, headers=self.sec_headers)

        soup = BeautifulSoup(response.content, "lxml")
        sic_table = soup.find('table', {'class': 'list'})
        sic_list = []
        for row in sic_table.find_all('tr')[1:]:
            sic_dict = {'SIC Code': None, 'Office': None, 'Industry Title': None}
            sic_dict['SIC Code'] = row.text.split('\n')[1]
            sic_dict['Office'] = row.text.split('\n')[2]
            sic_dict['Industry Title'] = row.text.split('\n')[3]
            sic_list.append(sic_dict)

        return sic_list

class TickerData(SECData):
    """Inherited from SECData class. Retrieves data from SEC Edgar database based on ticker.
    url is constructed based on the following: https://www.sec.gov/Archives/edgar/data/{cik}/{ascension_number}/{file_name}
    cik is the CIK number of the company = access via get_ticker_cik
    ascension_number is the accessionNumber column of filings_df
    file name for xml is always '{ticker}-{reportDate}.{extension}
    """

    def __init__(self, ticker: str, requester_company: str = 'Financial API', requester_name: str = 'API Caller', requester_email: str = 'apicaller@gmail.com', taxonomy: str = 'us-gaap',):
        super().__init__(requester_company, requester_name, requester_email, taxonomy)
        self.ticker = ticker.upper()
        self.cik = self.get_ticker_cik(self.ticker)
        self._submissions = self.get_submissions(self.cik)
        self._filings = None
        self._forms = None
        self._index = self.get_index(self.cik)
        self._filing_folder_urls = None
        self._filing_urls = None

    @property
    def submissions(self,) -> dict:
        if self._submissions is not None:
            self._submissions['filings'] = self.filings.replace({pd.NaT: None}).to_dict('records')
        return self._submissions

    @property
    def filings(self,) -> pd.DataFrame:
        if self._filings is None:
            self._filings = self.get_filings()
        return self._filings

    @property
    def latest_filing(self,) -> pd.DataFrame:
        return self.filings.iloc[0, :].to_dict() if len(self.filings) > 0 else None
    
    @property
    def latest_10Q(self,) -> pd.DataFrame:
        return self.filings.query("form == '10-Q'").iloc[0, :].to_dict() if len(self.filings.query("form == '10-Q'")) > 0 else None
    
    @property
    def latest_10K(self,) -> pd.DataFrame:
        return self.filings.query("form == '10-K'").iloc[0, :].to_dict() if len(self.filings.query("form == '10-K'")) > 0 else None
    
    @property 
    def latest_8K(self,) -> pd.DataFrame:
        return self.filings.query("form == '8-K'").iloc[0, :].to_dict() if len(self.filings.query("form == '8-K'")) > 0 else None

    @property
    def filing_folder_urls(self,) -> list:
        if self._filing_folder_urls is None:
            self._filing_folder_urls = self._get_filing_folder_urls()
        return self._filing_folder_urls

    @property
    def filing_urls(self,) -> list:
        if self._filing_urls is None:
            self._filing_urls = self.filings['file_url'].tolist()

        return self._filing_urls

    @property
    def forms(self,) -> list:
        if self._forms is None:
            self._forms = self.filings['form'].unique()
        return self._forms

    def _get_filing_folder_urls(self,) -> list:
        """Get filing folder urls from index dict.

        Args:
            index (dict): index dict from get_index method

        Returns:s
            filing_folder_urls (list): list of filing folder urls
        """

        filing_folder_urls = [self.BASE_SEC_URL + self._index['directory']['name'] + '/' + folder['name']
                              for folder in self._index['directory']['item'] if folder['type'] == 'folder.gif']
        return filing_folder_urls


    def _get_filing_urls(self,) -> list:
        """(DEPRECATED)
        ---The filing urls are implemented in the get_filings method.---
        
        Get filing urls from filing folder urls.

        Args:
            filing_folder_urls (list): list of filing folder urls

        Returns:
            filing_urls (list): list of filing urls to .txt files
        """
        filing_urls = []
        with trange(len(self.filing_folder_urls), desc=f'Instantiating filing urls for {self.ticker}...') as t:
            for i in t:
                self.scrape_logger.info(t)
                try:
                    soup = self.get_file_data(self.filing_folder_urls[i])
                    for link in soup.find_all('a'):
                        if link.get('href').endswith('.txt'):
                            filing_urls.append(
                                self.BASE_SEC_URL + link.get('href'))
                except Exception as e:
                    self.scrape_logger.error(
                        f'Failed to instantiate filing urls for {self.ticker}...')
                    self.scrape_logger.error(e)
                    t.write(
                        f'Failed to instantiate filing urls for {self.ticker}...')
                    continue
        return filing_urls
    

    def get_filings(self,) -> dict:
        """Get filings and urls to .txt from submissions dict.

        Args:
            submissions (dict): submissions dict from get_submissions method

        Returns:
            filings (dict): dictionary containing filings
        """
        self.scrape_logger.info(f'Making http request for {self.ticker} filings...')
        filings = self._submissions['filings']['recent']

        if len(self._submissions['filings']) > 1:
            self.scrape_logger.info(
                f'Additional filings found for {self.ticker}...')
            for file in self._submissions['filings']['files']:
                additional_filing = self.get_submissions(submission_file=file['name'])
                filings = {key: filings[key] + additional_filing[key] for key in filings.keys()}

        filings = pd.DataFrame(filings)
        # Convert reportDate, filingDate, acceptanceDateTime columns to datetime
        filings['reportDate'] = pd.to_datetime(filings['reportDate'])
        filings['filingDate'] = pd.to_datetime(filings['filingDate'])
        filings['acceptanceDateTime'] = pd.to_datetime(
            filings['acceptanceDateTime'])
        
        filings = filings.loc[~pd.isnull(filings['reportDate'])]
        
        # get folder url for each row
        filings['folder_url'] = self.BASE_DIRECTORY_URL + \
            self.cik + '/' + filings['accessionNumber'].str.replace('-', '')

        # get file url for each row
        filings['file_url'] = filings['folder_url'] + '/' + filings['accessionNumber'] + '.txt'
        
        return filings
    

    def get_file_data(self, file_url: str) -> BeautifulSoup:
        """Get file data from file url which can be retrieved by calling self.get_file_url method.

        Args:
            file_url (str): File url to retrieve data from on the SEC website

        Returns:
            data: File data as a BeautifulSoup object
        """
        data = self.rate_limited_request(
            url=file_url, headers=self.sec_headers)
        try:
            soup = BeautifulSoup(data.content, "lxml")
            self.scrape_logger.info(
                f'Parsed file data from {file_url} successfully.')
            return soup

        except Exception as e:
            self.scrape_logger.error(
                f'Failed to parse file data from {file_url}. Error: {e}')
            raise Exception(
                f'Failed to parse file data from {file_url}. Error: {e}')


    def search_tags(self, soup: BeautifulSoup, pattern: str) -> BeautifulSoup:
        """Search for tags in BeautifulSoup object.

        Args:
            soup (BeautifulSoup): BeautifulSoup object
            pattern (str): regex pattern to search for

        Returns:
            soup: BeautifulSoup object
        """
        return soup.find_all(re.compile(pattern))


    def search_context(self, soup: BeautifulSoup) -> pd.DataFrame:
        """Search for context in company .txt filing. 
        Context provides information about the entity, segment, and time period for facts in the filing.

        Args:
            soup (BeautifulSoup): BeautifulSoup object

        Returns:
            df: DataFrame containing context information with columns 
            {
                'contextId': str,
                'entity': str,
                'segment': str,
                'startDate': 'datetime64[ns]',
                'endDate': 'datetime64[ns]',
                'instant': 'datetime64[ns]'
            }
        """
        contexts = self.search_tags(soup, '^context$')
        dict_list = []
        columns = {'contextId': str, 'entity': str, 'segment': str,
                   'startDate': 'datetime64[ns]', 'endDate': 'datetime64[ns]', 'instant': 'datetime64[ns]'}
        for tag in contexts:
            temp_dict = {}
            temp_dict['contextId'] = tag.attrs.get('id')
            temp_dict['entity'] = tag.find("entity").text.split()[
                0] if tag.find("entity") is not None else None
            temp_dict['segment'] = tag.find("segment").text.strip(
            ) if tag.find("segment") is not None else None
            temp_dict['startDate'] = tag.find("startdate").text if tag.find(
                "startdate") is not None else None
            temp_dict['endDate'] = tag.find("enddate").text if tag.find(
                "enddate") is not None else None
            temp_dict['instant'] = tag.find("instant").text if tag.find(
                "instant") is not None else None
            dict_list.append(temp_dict)

        df = pd.DataFrame(dict_list, columns=columns.keys()).astype(columns)
        return df
    

    def search_linklabels(self, soup: BeautifulSoup) -> pd.DataFrame:
        """Search for link labels in company .txt filing. 
        Link labels provide information about the relationship between facts and their corresponding concepts.

        Args:
            soup (BeautifulSoup): BeautifulSoup object

        Returns:
            df: DataFrame containing link label information with columns 
            {
                'linkLabelId': str,
                'xlinkLabel': str,
                'xlinkRole': str,
                'xlinkType': str,
                'xlmnsXml': str,
                'xmlLang': str,
                'label': str
            }
        """
        links = self.search_tags(soup, '^link:label$')
        dict_list = []
        columns = {'linkLabelId': str, 'xlinkLabel': str, 'xlinkRole': str,
                    'xlinkType': str, 'xlmnsXml': str, 'xmlLang': str, 'label': str}

        for tag in links:
            temp_dict = {}
            temp_dict['linkLabelId'] = tag.attrs.get('id')
            temp_dict['xlinkLabel'] = tag.attrs.get('xlink:label')
            temp_dict['xlinkRole'] = tag.attrs.get('xlink:role')
            temp_dict['xlinkType'] = tag.attrs.get('xlink:type')
            temp_dict['xlmnsXml'] = tag.attrs.get('xmlns:xml')
            temp_dict['xmlLang'] = tag.attrs.get('xml:lang')
            temp_dict['label'] = tag.text if tag.text is not None else None
            dict_list.append(temp_dict)

        df = pd.DataFrame(dict_list, columns=columns.keys()).astype(columns)
        return df
    

    def search_facts(self, soup: BeautifulSoup) -> pd.DataFrame:
        """Search for facts in company .txt filing. 
        Facts provide the actual data values for the XBRL disclosures.

        Args:
            soup (BeautifulSoup): BeautifulSoup object

        Returns:
            df: DataFrame containing fact information with columns 
            {
                'factName': str,
                'contextRef': str,
                'decimals': int,
                'factId': str,
                'unitRef': str,
                'value': str
            }
        """
        facts = self.search_tags(soup, '^us-gaap:')
        dict_list = []
        columns = {'factName': str, 'contextRef': str, 'decimals': int, 'factId': str,
                   'unitRef': str, 'value': str}

        for tag in facts:
            temp_dict = {}
            temp_dict['factName'] = tag.name
            temp_dict['contextRef'] = tag.attrs.get('contextref')
            temp_dict['decimals'] = tag.attrs.get('decimals')
            temp_dict['factId'] = tag.attrs.get('id')
            temp_dict['unitRef'] = tag.attrs.get('unitref')
            temp_dict['value'] = tag.text
            dict_list.append(temp_dict)

        df = pd.DataFrame(dict_list, columns=columns.keys())
        df['factNameMerge'] = df['factName'].str.replace(':', '_')
        
        return df


    def get_metalinks(self, metalinks_url: str) -> pd.DataFrame:
        """Get metalinks from metalinks url.

        Args:
            metalinks_url (str): metalinks url to retrieve data from

        Returns:
            df: DataFrame containing metalinks information with columns 
            {
                'labelKey': str,
                'localName': str,
                'labelName': int,
                'terseLabel': str,
                'documentation': str,
            }
        """
        try:
            response = self.rate_limited_request(
                url=metalinks_url, headers=self.sec_headers).json()
        except Exception as e:
            self.scrape_logger.error(
                f'Failed to retrieve metalinks from {metalinks_url}. Error: {e}')
            raise Exception(
                f'Failed to retrieve metalinks from {metalinks_url}. Error: {e}')
        metalinks_instance = response['instance']
        instance_key = list(metalinks_instance.keys())[0]
        dict_list = []
        for i in metalinks_instance[instance_key]['tag']:
            dict_list.append(dict(labelKey=i.lower(), 
                localName=metalinks_instance[instance_key]['tag'][i].get('localname'),
                labelName=metalinks_instance[instance_key]['tag'][i].get('lang').get('en-us').get('role').get('label'),
                terseLabel=metalinks_instance[instance_key]['tag'][i].get('lang').get('en-us').get('role').get('terseLabel'),
                documentation=metalinks_instance[instance_key]['tag'][i].get('lang').get('en-us').get('role').get('documentation'),))
            
        df = pd.DataFrame.from_dict(dict_list)
        return df


    def __repr__(self) -> str:
        class_name = type(self).__name__
        main_attrs = ['ticker', 'cik', 'submissions', 'filings']
        available_methods = [method_name for method_name in dir(self) if callable(getattr(self, method_name)) and not method_name.startswith("_")]
        return f"""{class_name}({self.ticker})
    CIK: {self.cik}
    Latest filing: {self.latest_filing['filingDate'].strftime('%Y-%m-%d') if self.latest_filing else 'No filing found'} for Form {self.latest_filing['form'] if self.latest_filing else None}. Access via: {self.latest_filing['folder_url'] if self.latest_filing else None}
    Latest 10-Q: {self.latest_10Q['filingDate'].strftime('%Y-%m-%d') if self.latest_10Q else 'No filing found'}. Access via: {self.latest_10Q['folder_url'] if self.latest_10Q else None}
    Latest 10-K: {self.latest_10K['filingDate'].strftime('%Y-%m-%d') if self.latest_10K else 'No filing found'}. Access via: {self.latest_10K['folder_url'] if self.latest_10K else None}"""

# Get tags for filings

In [ ]:
aapl = TickerData(ticker='aapl')

In [ ]:
# Find reporting time frame of reportDate
soup = aapl.get_file_data(aapl.latest_10K['file_url'])
aapl.latest_10K['file_url']

## Search for tags

In [ ]:
context = aapl.search_context(soup)
facts = aapl.search_facts(soup)
labels = aapl.search_linklabels(soup)
metalinks = aapl.get_metalinks(aapl.latest_10K['folder_url'] + '/MetaLinks.json')

In [ ]:
facts_w_context = facts.merge(right=context, left_on='contextRef', right_on='contextId', how='left').merge(right=metalinks, left_on='factNameMerge', right_on='labelKey', how='left')

In [ ]:
facts_w_context

# Loop through all filings

In [3]:
sec = SECData()

In [ ]:
with trange(len(sec.cik['ticker'][:500]), desc='Instantiating tickers...',) as t:
    for item in t:
        ticker = sec.cik['ticker'].iloc[item]
        t.set_postfix(ticker=ticker)
        try:
            symbol = TickerData(ticker=ticker)
            sec.scrape_logger.info(f'\nInstantiated {symbol}...')
        except Exception as e:
            sec.scrape_logger.error(f'Failed to instantiate {ticker}...{e}')
            continue
        


        # for _, filing in symbol.filings.iterrows():
        #     if filing['form'] == '10-K':
        #         soup = symbol.get_file_data(filing['file_url'])

        #         context = symbol.search_context(soup)
        #         facts = symbol.search_facts(soup)
        #         metalinks = symbol.get_metalinks(filing['folder_url'] + '/MetaLinks.json')
                
        #         facts_w_context = facts.merge(right=context, left_on='contextRef', right_on='contextId', how='left').merge(right=metalinks, left_on='factNameMerge', right_on='labelKey', how='left')
                

In [ ]:
symbol = TickerData(ticker='V')
soup = symbol.get_file_data(symbol.filings.loc[symbol.filings['form'] == '10-K', 'file_url'].iloc[0])

contexts = symbol.search_tags(soup, '^context$')
dict_list = []
columns = {'contextId': str, 'entity': str, 'segment': str,
            'startDate': 'datetime64[ns]', 'endDate': 'datetime64[ns]', 'instant': 'datetime64[ns]'}
for tag in contexts:
    temp_dict = {}
    temp_dict['contextId'] = tag.attrs.get('id')
    temp_dict['entity'] = tag.find("entity").text.split()[
        0] if tag.find("entity").text is not None else None
    temp_dict['segment'] = tag.find("segment").text.strip(
    ) if tag.find("segment") is not None else None
    temp_dict['startDate'] = tag.find("startdate").text if tag.find(
        "startdate") is not None else None
    temp_dict['endDate'] = tag.find("enddate").text if tag.find(
        "enddate") is not None else None
    temp_dict['instant'] = tag.find("instant").text if tag.find(
        "instant") is not None else None
    dict_list.append(temp_dict)

In [ ]:
context = contexts[0]
data = {
    'id': context['id'],
    'entity': {
        'identifier': {
            'scheme': context.find('identifier')['scheme'],
            'value': context.find('identifier').text
        }
    },
    'period': {
        'startDate': context.find('startdate').text,
        'endDate': context.find('enddate').text
    }
}

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import XMLOutputParser
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessage,
    HumanMessagePromptTemplate,
)
from langchain.callbacks import get_openai_callback
import json

llm = ChatOpenAI(temperature=0)

parser = XMLOutputParser(tags=['id', 'entity', 'period'])
template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "You are a helpful assistant that parses XML files for a company's financial statements from the SEC Edgar database."
                "The XML content will be provided by the user."
                "You will parse the output and return it in the json format."
                "{format_instructions}"
            )
        ),
        HumanMessagePromptTemplate.from_template("{xml}"),
    ]
)

context_list = []
total_cost = 0
total_tokens = 0

llm = ChatOpenAI()
with trange(len(contexts[:]), desc='Scraping contexts...') as t:
    for i in t:
        with get_openai_callback() as cb:
            t.set_postfix(context=contexts[i].attrs.get('id'))
            output = llm(template.format_messages(format_instructions=parser.get_format_instructions(), xml=contexts[i]))
            total_cost += cb.total_cost
            total_tokens += cb.total_tokens
            context_list.append(json.loads(output.content))


# MongoDB Connection

In [73]:
from pymongo import MongoClient, ASCENDING, DESCENDING, GEO2D, GEOSPHERE, HASHED
from dotenv import load_dotenv
import os
from pprint import pprint
import datetime as dt
load_dotenv()

client = MongoClient(os.getenv('mongodb_sec'))
db = client.SECRawData
tickerdata = db.TickerData
tickerfilings = db.TickerFilings
sicdb = db.SICList
print(client.server_info())
print(db.list_collection_names())
print(tickerdata.index_information())
print(tickerfilings.index_information())

sec = SECData()


{'version': '6.0.10', 'gitVersion': '8e4b5670df9b9fe814e57cb5f3f8ee9407237b5a', 'modules': ['enterprise'], 'allocator': 'tcmalloc', 'javascriptEngine': 'mozjs', 'sysInfo': 'deprecated', 'versionArray': [6, 0, 10, 0], 'bits': 64, 'debug': False, 'maxBsonObjectSize': 16777216, 'storageEngines': ['devnull', 'ephemeralForTest', 'inMemory', 'queryable_wt', 'wiredTiger'], 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1696893337, 6), 'signature': {'hash': b'\xac\xb2\xdd\r\xdc\xcf1\xd7\x81\x1c\x1a\x8e0\x9aQ\xd8\x17+\xdb\xb5', 'keyId': 7252840401499848705}}, 'operationTime': Timestamp(1696893337, 6)}
['TickerFilings', 'TickerData']
{'_id_': {'v': 2, 'key': [('_id', 1)]}, 'cik_1': {'v': 2, 'key': [('cik', 1)], 'unique': True}}
{'_id_': {'v': 2, 'key': [('_id', 1)]}, 'cik_1': {'v': 2, 'key': [('cik', 1)], 'unique': True}, 'filings.form_1': {'v': 2, 'key': [('filings.form', 1)]}}


# Insert submission + filings of each ticker into mongodb TickerData collection

In [ ]:
failed_tickers = []
sic_dict = SECData().get_sic_list()

with trange(len(sec.cik_list['ticker'][:300]), desc='Instantiating ticker...',) as t:
    for item in t:
        ticker = sec.cik_list['ticker'].iloc[item]
        cik = sec.get_ticker_cik(ticker).lstrip('0')
        t.set_postfix(ticker=ticker, cik=cik)
        try:
            symbol = TickerData(ticker=ticker)
            symbol.submissions['lastUpdated'] = dt.datetime.now()
            symbol.submissions['office'] = sic_dict['Office'][sic_dict['SIC Code'].index(symbol.submissions['sic'])]
            filing_dict = {'ticker': ticker, 'cik': cik, 'filings': symbol.submissions['filings'], 'lastUpdated': dt.datetime.now()}
            sec.scrape_logger.info(f'{t}')
            sec.scrape_logger.info(f'\nInstantiated {symbol}...')
        except Exception as e:
            sec.scrape_logger.info(f'{t}')
            sec.scrape_logger.error(f'Failed to instantiate {ticker}...{e}')
            continue
        try:
            symbol.submissions.pop('filings')
            tickerdata.update_one({'cik': cik}, {'$set': symbol._submissions}, upsert=True)
            tickerfilings.update_one({'cik': cik}, {'$set': filing_dict}, upsert=True)
            sec.scrape_logger.info(f'Updated {ticker}...')
        except Exception as e:
            sec.scrape_logger.error(f'Failed to insert {ticker}...{e}')
            failed_tickers.append(ticker)
            sec.scrape_logger.info(f'Current failed tickers: {failed_tickers}...')
            continue

# Update SIC List

In [75]:
# store sic_dict in mongodb
sic_list = SECData().get_sic_list()
for i in range(len(sic_list)):
    sicdb.update_one({'SIC Code': sic_list[i]['SIC Code']}, {'$set': sic_list[i]}, upsert=True)

2023-10-10 01:19:11,403 - sec_scraper - INFO - Request successful at URL: https://www.sec.gov/corpfin/division-of-corporation-finance-standard-industrial-classification-sic-code-list


In [ ]:
# Update each record in tickerdata collection with Office field
ticker_list = [i for i in tickerdata.find({})]

with trange(tickerdata.count_documents({}), desc='Updating tickerdata...') as t:
    for i in t:
        t.set_postfix(i=i)

        try:
            cik = ticker_list[i]['cik']
            office = sicdb.find_one({'SIC Code': ticker_list[i]['sic']})['Office']
            tickerdata.update_one({'cik': cik}, {'$set': {'office': office}})
            t.write(f'Updated {cik}...')
        except Exception as e:
            t.write(f'Failed to update {cik}...{e}')
            continue
